In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

from scipy.stats import entropy


import lib as lib


In [2]:

# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [3]:
pk = np.array([1/5, 2/5, 2/5])  # fair coin
H = entropy(pk)
print(H)

1.0549201679861442


In [4]:


    
# test your function
list_labels=[1,2,0,1,2,0]
uniq_labels=[0,1,2]
print(lib.calculate_probabilities(list_labels,uniq_labels))
# this should print somehting like 0.33,0.33,0.33


{0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}


In [5]:



# test your function
list_probas=[1/5, 2/5, 2/5]
print(lib.calc_entropy_from_probabilities(list_probas))
# above should print 1.054...

1.0549201679861442


In [6]:



#test your function
old_entropy=1
new_entropies=[0,0.65]
count_items=[4,6]
print(lib.information_gain(old_entropy,new_entropies,count_items))
# above should print 0.61
    
    
    

0.61


In [7]:
num_feats=X_train.shape[1]
print(lib.initialize_weights(num_feats))

[0.98977562 0.66365385 0.92428663 0.33607177]


### Task4: PSO Implementation

#### Modified the entropy function to get a vector of entropies for n particles

In [8]:
### Below we just randomly assign 100 particles and see if we can find the global minimum.
### THis is just to check

In [9]:
params1=[lib.initialize_weights(X_train.shape[1]) for i in range(100)]# a vector of shape 100,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(100)]# a vector of shape 100
# use the np.random.uniform() function

# we have a list of 100 weight vectors (params1) and 100 thresholds (params2)
# convert them to array
params1=np.array(params1)
params2=np.array(params2)

print("Shape of params 1 (weights)",params1.shape)
print("Shape of params 2 (thresholds)",params2.shape)

Shape of params 1 (weights) (100, 4)
Shape of params 2 (thresholds) (100,)


In [10]:
z = lib.objective_fn_vector(params1, params2, X_train, y_train)
# Find the global minimum
param1_min = params1[z.argmin()] # use z.argmin()
param2_min = params2[z.argmin()] # use z.argmin()

print("param1_min",param1_min,"param2_min",param2_min)


param1_min [0.36309645 0.1253034  0.96045692 0.93853472] param2_min 0.5199850650639702


In [11]:
### Setting up the particles and other parameters now

In [12]:

# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 
# Create particles
n_particles = 20
np.random.seed(100)
params1=[lib.initialize_weights(X_train.shape[1]) for i in range(n_particles)] # a vector of shape n_particles,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(n_particles)]# a vector of shape n_particles
# use the np.random.uniform() function

params1=np.array(params1)
params2=np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)

params1 shape is  (20, 4) params2 shape is  (20,)


In [13]:
# define velocity of each weight of every particle
V_param1 = [lib.initialize_weights(X_train.shape[1])*0.1 for i in range(n_particles)] # shape is same as params1
# once again can use initialize_weights function

#define velocity of each threshold of every particle
V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])# shape is same as params2
# once again use np.random.uniform() function

# Initialize objective values
pbest = (params1,params2)
pbest_obj = lib.objective_fn_vector(params1, params2, X_train, y_train)
gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# note that gbest_obj should be the minimim of all pbest_obj

pbest obj value for 20 particles are as follows [0.67013703 0.82232957 0.64329013 0.70573338 0.73886477 0.82232957
 1.09729975 0.5237323  0.77244152 0.4620281  0.86703698 0.81919055
 1.09729975 0.81919055 0.73355763 0.74030523 0.82232957 0.73805779
 1.09729975 0.68309963]
gbest obj value among all 20 particles is as follows 0.4620281046196322


### the update function

In [14]:
def update():
    "Function to do one iteration of particle swarm optimization"
    global V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj
    # these have been already initialized in the previous cells
    
    # Update params
    r11,r12, r2 = np.random.rand(3)
    V_param1=w1*V_param1+c1*r11*(pbest[0] - params1)+ c2*r2*(gbest[0]-params1)
    V_param2=w2*V_param2+c1*r12*(pbest[1] - params2)+ c2*r2*(gbest[1]-params2)    
#     V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = lib.objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i]>=obj[i]:
            
            pbest[0][i]=params1[i] # update pbest[0][i] with value of params1[i]
            pbest[1][i]=params2[i] # update pbest[1][i] 
            pbest_obj[i]=obj[i]    # also update pbest_obj[i]

            
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()]) # update gbest to contain the best from params1 and params 2
    gbest_obj = pbest_obj.min() # update gbest to get the minimum of pbest_obj
 


In [15]:
for i in range(100):
    update()
print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min], lib.objective_fn(param1_min, param2_min, X_train, y_train)))


PSO found best solution at f((array([0.43418814, 0.27193862, 0.73106582, 1.09376118]), 0.8059805804680092))=0.4161039895073432
Global optimal at f([array([0.36309645, 0.1253034 , 0.96045692, 0.93853472]), 0.5199850650639702])=0.4620281046196322


### Try a different dataset (preferable binary classification)

In [16]:
# import some data to play with
#load the breast cancer dataset 
iris = datasets.load_breast_cancer()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [17]:
# initialize params1 and params2

params1=[lib.initialize_weights(X_train.shape[1]) for i in range(100)]# a vector of shape 100,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(100)]# a vector of shape 100
# use the np.random.uniform() function

# we have a list of 100 weight vectors (params1) and 100 thresholds (params2)
# convert them to array
params1=np.array(params1)
params2=np.array(params2)


print("Shape of params 1 (weights)",params1.shape)
print("Shape of params 2 (thresholds)",params2.shape)

Shape of params 1 (weights) (100, 30)
Shape of params 2 (thresholds) (100,)


In [18]:
z = lib.objective_fn_vector(params1, params2, X_train, y_train)
# Find the global minimum
param1_min = params1[z.argmin()] # use z.argmin()
param2_min = params2[z.argmin()] # use z.argmin()

print("param1_min",param1_min,"param2_min",param2_min)


param1_min [0.02185304 0.32185584 0.00369982 0.14475143 0.87311666 0.47985569
 0.01702597 0.78982985 0.3403445  0.00795039 0.74619156 0.83944554
 0.79193254 0.09661219 0.26514789 0.44572742 0.78537991 0.58004511
 0.06099062 0.68833783 0.10366424 0.27742988 0.05643949 0.96785451
 0.62472628 0.47809313 0.14646409 0.11329694 0.60353699 0.17073787] param2_min 0.7972951581704367


In [19]:

# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 
# Create particles
n_particles = 20
np.random.seed(100)
params1=[lib.initialize_weights(X_train.shape[1]) for i in range(n_particles)] # a vector of shape n_particles,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(n_particles)]# a vector of shape n_particles
# use the np.random.uniform() function

params1=np.array(params1)
params2=np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)

params1 shape is  (20, 30) params2 shape is  (20,)


In [20]:
# define velocity of each weight of every particle
V_param1 = [lib.initialize_weights(X_train.shape[1])*0.1 for i in range(n_particles)] # shape is same as params1
# once again can use initialize_weights function

#define velocity of each threshold of every particle
V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])# shape is same as params2
# once again use np.random.uniform() function

# Initialize objective values
pbest = (params1,params2)
pbest_obj = lib.objective_fn_vector(params1, params2, X_train, y_train)
gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# note that gbest_obj should be the minimim of all pbest_obj

pbest obj value for 20 particles are as follows [0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461
 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461
 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461]
gbest obj value among all 20 particles is as follows 0.6643460978641622


In [21]:
def update():
    "Function to do one iteration of particle swarm optimization"
    global V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj
    # these have been already initialized in the previous cells
    
    # Update params
    r11,r12, r2 = np.random.rand(3)
    V_param1=w1*V_param1+c1*r11*(pbest[0] - params1)+ c2*r2*(gbest[0]-params1)
    V_param2=w2*V_param2+c1*r12*(pbest[1] - params2)+ c2*r2*(gbest[1]-params2)    
#     V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = lib.objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i]>=obj[i]:
            
            pbest[0][i]=params1[i] # update pbest[0][i] with value of params1[i]
            pbest[1][i]=params2[i] # update pbest[1][i] 
            pbest_obj[i]=obj[i]    # also update pbest_obj[i]

            
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()]) # update gbest to contain the best from params1 and params 2
    gbest_obj = pbest_obj.min() # update gbest to get the minimum of pbest_obj
 


In [22]:
for i in range(100):
    update()
print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min], lib.objective_fn(param1_min, param2_min, X_train, y_train)))


PSO found best solution at f((array([0.59729898, 5.47102916, 0.47442985, 0.86846664, 0.00599996,
       0.13605053, 0.69659518, 1.03239899, 0.21786187, 0.74464396,
       1.00613508, 0.86428376, 0.19965348, 0.1385734 , 0.49490825,
       0.99910188, 0.87907455, 0.24718113, 0.82794612, 0.43561366,
       0.65338515, 0.95339072, 0.91566639, 0.41604149, 0.80216502,
       0.44830097, 0.24210009, 0.27194124, 0.86069922, 0.01889471]), 0.6462061717794414))=0.6643460978641622
Global optimal at f([array([0.02185304, 0.32185584, 0.00369982, 0.14475143, 0.87311666,
       0.47985569, 0.01702597, 0.78982985, 0.3403445 , 0.00795039,
       0.74619156, 0.83944554, 0.79193254, 0.09661219, 0.26514789,
       0.44572742, 0.78537991, 0.58004511, 0.06099062, 0.68833783,
       0.10366424, 0.27742988, 0.05643949, 0.96785451, 0.62472628,
       0.47809313, 0.14646409, 0.11329694, 0.60353699, 0.17073787]), 0.7972951581704367])=0.44322120624369
